In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
census_starter = pd.read_csv('census_starter.csv')
sample_submission = pd.read_csv('sample_submission.csv')
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

In [3]:
train = train.merge(census_starter, how='left', on='cfips')

In [4]:
train['last_month_data'] = train.groupby(['cfips'])['microbusiness_density'].shift(1)

In [5]:
train['year'] = train.first_day_of_month.apply(lambda x: int(x.split('-')[0]))

In [6]:
train['pct_bb_1'] = train.year.apply(lambda y: 'pct_bb_'+ str(y-2))
train['pct_bb_2'] = train.year.apply(lambda y: 'pct_bb_'+ str(y-1))

train['pct_college_1'] = train.year.apply(lambda y: 'pct_college_'+ str(y-2))
train['pct_college_2'] = train.year.apply(lambda y: 'pct_college_'+ str(y-1))

train['pct_foreign_born_1'] = train.year.apply(lambda y: 'pct_foreign_born_'+ str(y-2))
train['pct_foreign_born_2'] = train.year.apply(lambda y: 'pct_foreign_born_'+ str(y-1))

train['pct_it_workers_1'] = train.year.apply(lambda y: 'pct_it_workers_'+ str(y-2))
train['pct_it_workers_2'] = train.year.apply(lambda y: 'pct_it_workers_'+ str(y-1))

train['median_hh_inc_1'] = train.year.apply(lambda y: 'median_hh_inc_'+ str(y-2))
train['median_hh_inc_2'] = train.year.apply(lambda y: 'median_hh_inc_'+ str(y-1))

In [7]:
train['pct_bb_1'] = train.apply(lambda x: x[x['pct_bb_1']], axis=1)
train['pct_bb_2'] = train.apply(lambda x: x[x['pct_bb_2']], axis=1)

train['pct_college_1'] = train.apply(lambda x: x[x['pct_college_1']], axis=1)
train['pct_college_2'] = train.apply(lambda x: x[x['pct_college_2']], axis=1)

train['pct_foreign_born_1'] = train.apply(lambda x: x[x['pct_foreign_born_1']], axis=1)
train['pct_foreign_born_2'] = train.apply(lambda x: x[x['pct_foreign_born_2']], axis=1)

train['pct_it_workers_1'] = train.apply(lambda x: x[x['pct_it_workers_1']], axis=1)
train['pct_it_workers_2'] = train.apply(lambda x: x[x['pct_it_workers_2']], axis=1)

train['median_hh_inc_1'] = train.apply(lambda x: x[x['median_hh_inc_1']], axis=1)
train['median_hh_inc_2'] = train.apply(lambda x: x[x['median_hh_inc_2']], axis=1)

In [8]:
train.drop(census_starter.drop(['cfips'], axis=1), axis=1, inplace=True)

In [9]:
train = train[~train.last_month_data.isna()]

In [10]:
train['microbusiness_density_precent'] = train.microbusiness_density/train.last_month_data

In [11]:
remove_fcips = train[(train['microbusiness_density_precent'] > 2) | (train['microbusiness_density_precent'] < 0.5)][['cfips']]

In [12]:
train = train[~train['cfips'].isin(remove_fcips.cfips.values)]

# tf

In [13]:
from sklearn import preprocessing

In [14]:
def prepare_date1(train, n_steps, batch_size, shift):

    x = None
    # 先取前1500，再大我的電腦會kernal died
    for i in train.cfips.unique()[:2]:
#         print(i)
        data = train[train.cfips == i][['last_month_data', 'pct_bb_1', 'pct_bb_2', 'pct_college_1', 'pct_college_2', 'pct_foreign_born_1', 'pct_foreign_born_2', 'pct_it_workers_1', 'pct_it_workers_2', 'median_hh_inc_1', 'median_hh_inc_2']]
        y = train[train.cfips == i][['microbusiness_density_precent']].reset_index(drop=True)
        
        minmax = preprocessing.MinMaxScaler()
        data = pd.DataFrame(minmax.fit_transform(data))
        data['y'] = y
        
        dataset = tf.data.Dataset.from_tensor_slices(data)
        window_length = n_steps + 10
        dataset = dataset.window(window_length, shift=1, drop_remainder=True)
        dataset = dataset.flat_map(lambda window: window.batch(window_length))

        if x == None:
            x = dataset
        else:
            x = x.concatenate(dataset)

    x = x.batch(batch_size)
    x1 = x.map(lambda windows: (windows[:, :n_steps, 0:1], windows[:, :n_steps, 1:-1], windows[:, -10:, -1]))
#     x2 = x.map(lambda windows: (windows[:, :n_steps, 1:-1]))
#     y = x.map(lambda windows: (windows[:, -10:, -1]))
    
    

    x1 = x1.shuffle(10000, seed=1)
#     x2 = x2.shuffle(10000, seed=1)
#     y = y.shuffle(10000, seed=1)
    
#     for a, b, c in zip(x1, x2, y):
#         stack_x1 = np.stack(a.numpy()) 
#         stack_x2 = np.stack(b.numpy()) 
#         stack_y = np.stack(c.numpy())  
    for a, b, c in x1:
        stack_x1 = np.stack(a.numpy()) 
        stack_x2 = np.stack(b.numpy()) 
        stack_y = np.stack(c.numpy())  
#     x1 = x1.prefetch(1)
#     x2 = x2.prefetch(1)
#     x1 = x.map(lambda windows: (windows[:, :n_steps, :-1], windows[:, -10:, -1]))
    return stack_x1, stack_x2, stack_y

In [15]:
x1, x2, y = prepare_date1(train, n_steps=20, batch_size=64, shift=1)

In [16]:
he_avg_init = tf.keras.initializers.VarianceScaling(scale=2, mode='fan_avg', distribution='uniform')

In [17]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [18]:
from tensorflow.keras import Input, Model
from tensorflow.keras.losses import MeanAbsolutePercentageError
from tensorflow.keras.layers import Dense, Lambda, concatenate, Bidirectional, LSTM, TimeDistributed, BatchNormalization, Concatenate
from tensorflow.keras.layers import LayerNormalization
from tensorflow.keras import backend

In [103]:
inputs = Input(shape=(None, 1))
inputs_opendata_reshape = tf.reshape(Input(shape=(None, 10)), [-1, 10])

x = Bidirectional(LSTM(64, return_sequences=True))(inputs)
x = LayerNormalization()(x)
x = Bidirectional(LSTM(64, return_sequences=False))(x)
x = Dense(64, activation='elu', kernel_initializer=he_avg_init)(x)
x = BatchNormalization()(x)

x_concat = Concatenate(axis=-1)([x, inputs_opendata_reshape])
x_concat = Dense(32, activation='elu', kernel_initializer=he_avg_init)(x_concat)
x_concat = Dense(10)(x_concat)
model = Model(inputs=[inputs, inputs_opendata_reshape], outputs=x_concat)

In [104]:
# model.compile(loss=MeanAbsolutePercentageError(), optimizer="adam")
model.compile(loss='mse', optimizer="adam")

In [105]:
# combined_dataset = tf.data.Dataset.zip((x,y))
# combined_dataset = combined_dataset.batch(1)

In [106]:
# dataset = tf.data.Dataset.from_tensors(((a,b), c))
# model.fit(dataset, epochs=10, steps_per_epoch=4)

In [107]:
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_29 (InputLayer)          [(None, None, 1)]    0           []                               
                                                                                                  
 bidirectional_24 (Bidirectiona  (None, None, 128)   33792       ['input_29[0][0]']               
 l)                                                                                               
                                                                                                  
 layer_normalization_12 (LayerN  (None, None, 128)   256         ['bidirectional_24[1][0]']       
 ormalization)                                                                                    
                                                                                            

In [108]:
dataset_12 = tf.data.Dataset.from_tensor_slices((x1, x2))
dataset_label = tf.data.Dataset.from_tensor_slices(y)

dataset = tf.data.Dataset.zip((dataset_12, dataset_label))
model.fit(dataset, epochs=10, steps_per_epoch=4)

Epoch 1/10


ValueError: in user code:

    File "C:\Users\rocker\anaconda3\envs\tf2\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\rocker\anaconda3\envs\tf2\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\rocker\anaconda3\envs\tf2\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\rocker\anaconda3\envs\tf2\lib\site-packages\keras\engine\training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\rocker\anaconda3\envs\tf2\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "C:\Users\rocker\anaconda3\envs\tf2\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        l

In [115]:
x1.shape

(18, 20, 1)

In [35]:
# def generator():
#     for s1, s2, l in zip(x1, x2, y):
#         yield {"input_1": s1, "input_2": s2}, l

# dataset = tf.data.Dataset.from_generator(generator, output_types=({"input_5": tf.float64, "input_6": tf.float64}, tf.float64))


In [36]:
def _input_fn():
  dataset = tf.data.Dataset.from_tensor_slices(({"input_1": x1, "input_2": x2}, y))
  dataset = dataset.batch(2, drop_remainder=True)
  return dataset

In [37]:
# a = tf.data.Dataset.from_tensors(x1)
# b = tf.data.Dataset.from_tensors(x2)
# c = tf.data.Dataset.from_tensors(y)

In [38]:
# dataset = tf.data.Dataset.from_tensor_slices({"input_1": x1, "input_2": x2}, y)

In [116]:
# history = model.fit(_input_fn() , epochs=3)